In [1]:
!pip install ipynb

In [13]:
from ipynb.fs.full. scraping import *
import csv 
import time

In [14]:
headers = "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) snap Chromium/83.0.4103.106 Chrome/83.0.4103.106 Safari/537.36"
training_keywords = {'করোনা', 'কোভিড-১৯', 'উহান-ভাইরাস', 'নতুন ভ্যারিয়েন্টে করনা'}

In [16]:
def store_in_csv(item):
    with open('data.csv', 'a+', newline='') as file:
        file_header = ['Tittle', 'link']
        writer = csv.writer(file, file_header)
        writer.writerow(item)
        

In [17]:
def find_date_time(string):
    items = string.find_all('time')
    if len(items)>0:
        for i in items:
            return i.get_text().strip()
    return ""

In [25]:
def print_only_corona_news(links):
    news_count = 0
    for link in links:
        news_link = get_valid_link(str(link.get('href')))
        title = link.get_text().strip()
        if news_link is None or len(title) <=0:
            continue
        title_words = set(title.split())
        
        if training_keywords.intersection(title_words):
            store_in_csv([title, news_link])
            news_count +=1
        else:
            metadata = scrap_url(news_link, headers).find_all('meta')
            for tag in metadata:
                if 'name' in tag.attrs.keys() and tag.attrs['name'].strip().lower() in ['keywords', 'description']:
                    sample_keywords = set(tag.attrs['content'].strip().split())
                    if training_keywords.intersection(sample_keywords):
                        news_count +=1
                        store_in_csv([title, news_link])
                        
    return news_count

                

In [26]:
if __name__=='__main__':
    start_time = time.time()
    total_news =0
    with open('data.csv', 'w', newline='') as data:
        writer = csv.DictWriter(data, fieldnames = ['Title', 'Link'])
        writer.writeheader()
        
    website_list = ["https://jamuna.tv", "https://bangla.bdnews24.com", "https://somoynews.tv", "https://prothomalo.com"]  # add target website URL
    for web_url in website_list:
        soup = scrap_url(web_url, headers)
        all_links = soup.find_all('a')
        total_news +=print_only_corona_news(all_links)
    print(f'Total {total_news} news collected')
    print(f'Total time spent: {time.time() - start_time} seconds')

HTTPSConnectionPool(host='bangla.bdnews24.com', port=443): Max retries exceeded with url: /media_bn/article1899981.bdnews?Mozilla/5.0%20(X11;%20Linux%20x86_64)%20AppleWebKit/537.36%20(KHTML,%20like%20Gecko)%20snap%20Chromium/83.0.4103.106%20Chrome/83.0.4103.106%20Safari/537.36 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fad4d455250>: Failed to establish a new connection: [Errno 110] Connection timed out'))
Total 12 news collected
Total time spent: 944.5629053115845 seconds


In [27]:
import pandas as pd
df = pd.read_csv('data.csv')

In [28]:
df.head()

,Title,Link
0,পটুয়াখালীতে ১৫ হাজার মানুষ করোনা টিকার দ্বিতীয়...,https://jamuna.tv/news/220576
1,করোনা সচেতনতায় গুগলের ডুডল,https://jamuna.tv/news/215580
2,করোনা সচেতনতায় গুগলের ডুডল,https://jamuna.tv/news/217424
3,যেসব কারণে করোনা থেকে সেরে ওঠে খিচুড়ি খাবেন,https://jamuna.tv/news/225078
4,করোনা দুর্যোগ: ভারতীয় মিডিয়া বনাম বাংলাদেশি ...,https://jamuna.tv/news/217120
